## Use a serverless Spark compute

You should have an attached Synapse Spark pool available in your workspace. Please see documentation page: [Attach and manage a Synapse Spark pool in Azure Machine Learning (preview)](https://learn.microsoft.com/azure/machine-learning/how-to-manage-synapse-spark-pool) for more details.

**Note** - To ensure successful execution of Spark job, the identity being used for the Spark job should be assigned **Contributor** and **Storage Blob Data Contributor** roles on the Azure storage account used for data input and output.

In [ ]:
from azure.ai.ml import MLClient, spark, Input, Output
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Environment

subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

In [ ]:
import yaml

contents = ""
with open("../project/env/online.yml", "r") as stream:
    try:
        contents = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [ ]:
spark_job = spark(
    display_name="featurestore_sample_test",
    code="../",
    entry={"file": "automation-test/featurestore_vnet_job.py"},
    driver_cores=1,
    driver_memory="1g",
    executor_cores=1,
    executor_memory="1g",
    executor_instances=1,
    resources={
        "instance_type": "Standard_E8S_V3",
        "runtime_version": "3.4.0",
    },
    conf={"spark.synapse.library.python.env": contents},
)

returned_spark_job = ml_client.jobs.create_or_update(spark_job)

print(returned_spark_job.id)
# Wait until the job completes
ml_client.jobs.stream(returned_spark_job.name)